In [ ]:
import json
import os
import requests
from bs4 import BeautifulSoup
from src.utils import *
from src.database_structure import *
from src.const_parameters import *
from src.auth import *

In [ ]:
def enrichItem(session, path):
    item =  TorrentData.from_json(path)
    getStr = f"{BASE_ADDRESS_URI}/torrent/{item.guid}/"
    response = session.get(getStr)
    if response.status_code == 200:
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')

        for div in soup.find_all('img', {'style':f'view-transition-name: poster-{item.guid};'}):
            item.ref_image = div.attrs['src']
            if(len(item.ref_image) > 0):
                if(item.ref_image.startswith("//")):
                    item.ref_image = "https:" +  item.ref_image
            else:
                print(f"Image no found for guid: {item.guid}")
                
        for div in soup.find_all('div', {'style':f'view-transition-name: description-{item.guid}'}):
            for ch in div.children:
                item.descriptions.append(str(ch))

        item.to_json(path)
        print(f"Enrich {path} success!")
    else:
        print(f"Enrich {path} failed!")

In [ ]:
def downloadTorrent(session, path):
    item =  TorrentData.from_json(path)
    response = session.get(f"{BASE_ADDRESS_URI}/torrent/{item.guid}/download/")
    with open(change_extension(path, ".torrent"), 'wb') as file:
        file.write(response.content)

In [ ]:
def downloadPoster(session, path):
    item =  TorrentData.from_json(path)
    response = session.get(item.ref_image)
    base_name, extension = os.path.splitext(item.ref_image)
    with open(change_extension(path, extension), 'wb') as file:
        file.write(response.content)

# Выполнение

In [ ]:
if(__name__ == "__main__"):
    loginData = readLogindata("credentials.json")
    session = createSessionWithLogin(loginData)
    
    files = get_json_files(FOLDER_DATABASE)
    for path in files:
        enrichItem(session, path)
        downloadPoster(session, path)
        downloadTorrent(session, path)
